In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pandas
import sqlalchemy
from dotenv import load_dotenv
import os
import shutil

# Configuração do Selenium
options = Options()
options.add_argument("--headless")  # Executa em segundo plano
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
print("Ambiente pronto")

Ambiente pronto


In [62]:
load_dotenv()

email = os.getenv("LINKEDIN_EMAIL")
senha = os.getenv("LINKEDIN_PASSWORD")

In [68]:
driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/in/walter-gonzaga/')

In [69]:
time.sleep(5)
driver.find_element(By.CSS_SELECTOR,'#main-content > div > form > p > button').click()

In [70]:
input_email = driver.find_element(By.ID,'session_key')
input_email.click()
input_email.send_keys(email)

input_senha = driver.find_element(By.ID,'session_password')
input_senha.click()
input_senha.send_keys(senha)

driver.find_element(By.CSS_SELECTOR,'#main-content > div > div.authwall-sign-in-form > form > div.flex.justify-between.sign-in-form__footer--full-width > button').click()


In [71]:
driver.get('https://www.linkedin.com/analytics/creator/content/?timeRange=past_365_days')

### Coletar URLS de posts - Historico

In [79]:
lista_links = []
links = driver.find_elements(By.CSS_SELECTOR, "a.member-analytics-addon__cta-item-with-secondary-anchor")

for a in links:
    if "impressões" in a.get_attribute("aria-label").lower():
        link = a.get_attribute("href")
        print(link)
        lista_links.append(link)
        


https://www.linkedin.com/analytics/post-summary/urn:li:activity:7424076450837864448
https://www.linkedin.com/analytics/post-summary/urn:li:activity:7414630220768702464
https://www.linkedin.com/analytics/post-summary/urn:li:activity:7321577776015134722
https://www.linkedin.com/analytics/post-summary/urn:li:activity:7400503920122490880
https://www.linkedin.com/analytics/post-summary/urn:li:activity:7317529845620441088
https://www.linkedin.com/analytics/post-summary/urn:li:activity:7405214983263055872
https://www.linkedin.com/analytics/post-summary/urn:li:activity:7369710710492266499
https://www.linkedin.com/analytics/post-summary/urn:li:activity:7407049141337538560
https://www.linkedin.com/analytics/post-summary/urn:li:activity:7397650027701108736
https://www.linkedin.com/analytics/post-summary/urn:li:activity:7425146667781443584
https://www.linkedin.com/analytics/post-summary/urn:li:activity:7413905911544602624
https://www.linkedin.com/analytics/post-summary/urn:li:activity:740629071071

In [87]:
driver.find_element(By.XPATH,'//*[@id="ember482"]/span').click()

In [ ]:
for link in lista_links:
    driver.get(link)
    time.sleep(5)
    wait = WebDriverWait(driver, 10)

    botao_exportar = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//button[.//span[text()='Exportar']]")
        )
    )

    botao_exportar.click()
    time.sleep(3)

In [ ]:
# PREFIXO dos arquivos
prefixo = "PostAnalytics_WalterGonzaga"

# Pasta Downloads (Windows padrão)
downloads = os.path.join(os.environ["USERPROFILE"], "Downloads")

# Pasta destino
destino = r"C:\Users\Walter\Documents\Comunidados\scraper_linkedin_base_metricas_p_analytics_e_ML\arquivos"

# Garante que a pasta existe
os.makedirs(destino, exist_ok=True)

movidos = 0

for arquivo in os.listdir(downloads):
    if arquivo.startswith(prefixo):
        origem_arquivo = os.path.join(downloads, arquivo)
        destino_arquivo = os.path.join(destino, arquivo)

        shutil.move(origem_arquivo, destino_arquivo)
        print(f"Movido: {arquivo}")
        movidos += 1

print(f"\nTotal movidos: {movidos}")


Movido: PostAnalytics_WalterGonzaga_7418406205393600513.xlsx
Movido: PostAnalytics_WalterGonzaga_7424076449646731264 (1).xlsx
Movido: PostAnalytics_WalterGonzaga_7424076449646731264.xlsx

Total movidos: 3
